# 本任务主要是进行特征工程

In [9]:
import pandas as pd

In [10]:
# 导入数据集
train=pd.read_csv("../data/train_data.csv")
test=pd.read_csv("../data/test_data.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096 entries, 0 to 3095
Data columns (total 82 columns):
low_volume_percent                            3096 non-null float64
middle_volume_percent                         3096 non-null float64
take_amount_in_later_12_month_highest         3096 non-null int64
trans_amount_increase_rate_lately             3096 non-null float64
trans_activity_month                          3096 non-null float64
trans_activity_day                            3096 non-null float64
transd_mcc                                    3096 non-null float64
trans_days_interval_filter                    3096 non-null float64
trans_days_interval                           3096 non-null float64
regional_mobility                             3096 non-null float64
student_feature                               3096 non-null float64
repayment_capability                          3096 non-null int64
is_high_user                                  3096 non-null int64
number_of_trans

In [12]:
# 将训练集和测试集的特征与标签分开
train_y=train['status']
train_x=train.drop('status',axis=1)
test_y=test['status']
test_x=test.drop('status',axis=1)

In [13]:
features=pd.concat([train_x,test_x])

In [14]:
# 通过统计方法，计算历史总共借款，和借款的成功还款比例
features['historical_suc_fail_fee'] = (features['history_suc_fee']+features['history_fail_fee'])
features['historical_suc_rate']=features['history_suc_fee']/(features['historical_suc_fail_fee']+0.01)#smoothing
features['historical_fail_rate']=features['history_fail_fee']/(features['historical_suc_fail_fee']+0.01)#smoothing


In [16]:
# 归一化
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics=[]
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics.append(i)
str(numerics)

"['low_volume_percent', 'middle_volume_percent', 'take_amount_in_later_12_month_highest', 'trans_amount_increase_rate_lately', 'trans_activity_month', 'trans_activity_day', 'transd_mcc', 'trans_days_interval_filter', 'trans_days_interval', 'regional_mobility', 'student_feature', 'repayment_capability', 'is_high_user', 'number_of_trans_from_2011', 'first_transaction_time', 'historical_trans_amount', 'historical_trans_day', 'rank_trad_1_month', 'trans_amount_3_month', 'avg_consume_less_12_valid_month', 'abs', 'top_trans_count_last_1_month', 'avg_price_last_12_month', 'avg_price_top_last_12_valid_month', 'reg_preference_for_trad', 'trans_top_time_last_1_month', 'trans_top_time_last_6_month', 'consume_top_time_last_1_month', 'consume_top_time_last_6_month', 'cross_consume_count_last_1_month', 'trans_fail_top_count_enum_last_1_month', 'trans_fail_top_count_enum_last_6_month', 'trans_fail_top_count_enum_last_12_month', 'consume_mini_time_last_1_month', 'max_cumulative_consume_later_1_month',

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(features[numerics].values)
features[numerics] = scaler.transform(features[numerics].values)

In [20]:
#使用.get_dummies()方法对特征矩阵进行类似“坐标投影”操作。获得在新空间下的特征表达。
final_features = pd.get_dummies(features)
final_features

,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,regional_mobility,...,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,historical_suc_fail_fee,historical_suc_rate,historical_fail_rate
0,-0.300790,0.621891,2.252947,-0.014732,-1.448001,-1.306635,-1.038731,0.095038,0.336382,1.481320,...,-0.080985,-1.264844,-1.249729,-0.926247,-0.886276,-0.557618,1.709946,-1.059771,0.494235,-0.496026
1,-0.300790,0.621891,-0.018757,-0.020529,-1.244616,-1.324287,-2.388106,2.791271,0.336382,-0.765429,...,0.199285,-0.928090,-0.955948,-0.884130,-1.009926,-0.077810,-0.503430,-0.509042,0.893121,-0.892044
2,0.222766,0.479724,3.001860,-0.018972,0.229923,-0.765300,-0.139149,-0.567970,2.183803,1.481320,...,0.619691,-0.928090,-0.955948,-0.547193,0.067599,-0.424338,1.784976,-0.641217,-0.509292,0.509432
3,0.222766,0.479724,-0.493068,-0.018068,0.992616,-1.194838,-0.813836,0.006637,-0.341006,-0.765429,...,0.059150,0.755677,1.100513,-0.378724,-0.664235,0.028814,-0.953609,0.107774,-0.694294,0.695197
4,-0.300790,0.621891,-0.368250,-0.002904,-1.092077,-0.259269,0.085747,-0.479569,-0.341006,-1.888804,...,0.129218,-0.254583,-0.074608,-0.757779,-0.714048,0.055470,-1.009881,-0.729334,0.932474,-0.931876
5,-0.039012,-0.302192,0.006207,-0.019695,0.382462,-1.030083,0.085747,0.227640,-0.341006,-0.765429,...,-5.336052,-1.601597,-1.543509,-1.136833,-1.416206,-0.584274,-0.672247,-1.125858,0.248978,-0.252107
6,-0.039012,-0.870859,2.452657,-0.020403,0.992616,0.111428,0.985330,0.095038,-0.279425,0.357945,...,0.269353,0.082170,-0.074608,0.042447,-0.070183,-0.584274,-0.747277,0.460241,-0.819457,0.820456
7,-0.039012,-0.088943,0.755120,-0.020626,0.535000,-0.488748,1.660017,0.095038,0.090059,0.357945,...,0.409488,-0.254583,-0.368388,-0.041787,0.005774,-0.530962,0.471956,-0.685275,0.781304,-0.780652
8,-0.562567,-0.017859,-0.493068,-0.019597,0.992616,-0.035674,-0.364044,0.095038,0.521124,-0.765429,...,0.199285,0.755677,0.512953,3.159117,1.869364,-0.317714,-0.916094,0.151833,-0.401628,0.402696
9,0.746321,0.337557,0.056135,-0.020515,0.229923,-0.488748,-0.364044,-0.435368,-0.217844,0.357945,...,-0.641525,-1.264844,-1.249729,-0.968364,-0.992262,3.120908,-0.859822,-1.147888,1.306695,-1.309902


In [21]:
train_x=features.iloc[:len(train_y),:]#y是列向量，存储了训练数据中的房价列信息。截取后得到的X阵的维度是len(y)*(final_features的列数)。
test_x=features.iloc[len(train_y):,:]

## 使用IV来进行特征选择

In [25]:
import numpy as np
def CalcIV(Xvar,Yvar,k):
    if len(np.unique(Xvar))<=10:
        N_0=np.sum(Yvar==0)+len(np.unique(Xvar))*0.01# 防止除数为0，作平滑处理
        N_1=np.sum(Yvar==1)+len(np.unique(Xvar))*0.01
        N_0_group=np.zeros(np.unique(Xvar).shape)
        N_1_group=np.zeros(np.unique(Xvar).shape)
    
        for i in range(len(np.unique(Xvar))):       
            N_0_group[i] = len(Yvar[(Xvar==np.unique(Xvar)[i])&(Yvar==0)])+0.01
            N_1_group[i] = len(Yvar[(Xvar==np.unique(Xvar)[i])&(Yvar==1)])+0.01
        iv = np.sum((N_0_group/N_0-N_1_group/N_1)*np.log((N_0_group/N_0)/(N_1_group/N_1)))
    # 连续型特征，分成k组
    else:
        N_0=np.sum(Yvar==0)+k*0.01
        N_1=np.sum(Yvar==1)+k*0.01
        N_0_group=np.zeros(k)
        N_1_group=np.zeros(k)
        for i in range(k-1):
            n = len(np.unique(Xvar))//k
            N_0_group[i] = len(Yvar[(Xvar>=np.unique(Xvar)[i*n])&(Xvar<np.unique(Xvar)[(i+1)*n])&(Yvar==0)])+0.01
            N_1_group[i] = len(Yvar[(Xvar>=np.unique(Xvar)[i*n])&(Xvar<np.unique(Xvar)[(i+1)*n])&(Yvar==1)])+0.01
        N_0_group[k-1] = len(Yvar[(Xvar>=np.unique(Xvar)[(k-1)*n])&(Yvar==0)])+0.01
        N_1_group[k-1] = len(Yvar[(Xvar>=np.unique(Xvar)[(k-1)*n])&(Xvar<np.unique(Xvar)[(i+1)*n])&(Yvar==1)])+0.01
        iv = np.sum((N_0_group/N_0-N_1_group/N_1)*np.log((N_0_group/N_0)/(N_1_group/N_1)))
    return iv

def caliv_batch(df,Yvar,k=10):
    ivlist=[]
    for col in df.columns:
        iv=CalcIV(df[col],Yvar,k)
        ivlist.append(iv)
    names=list(df.columns)
    iv_df=pd.DataFrame({'Var':names,'IV':ivlist},columns=['Var','IV'])

    return iv_df,ivlist
im_iv, ivl = caliv_batch(train_x,train_y)

In [26]:
ivl[:5]

[0.06036614448820549,
 8.299065237846454,
 0.1282208872250961,
 0.18112969573700177,
 4.228639751339617]

In [30]:
# 查看各个特征的IV值，按照从大到小进行排列
sorted(im_iv['IV'],reverse=True)

[8.299065237846454,
 7.188909990255593,
 4.228639751339617,
 2.517840192693167,
 2.314724141099322,
 1.0725559733164254,
 1.0725559733164254,
 0.9569239947456017,
 0.9231383072642612,
 0.8774968287151022,
 0.8694720451523579,
 0.8622241558702736,
 0.7422906558110307,
 0.7239029039050044,
 0.7168065908012751,
 0.6603892807963205,
 0.5502741581653209,
 0.545656038034965,
 0.5385891623237261,
 0.5346706849585943,
 0.5224509380999404,
 0.4858262532751586,
 0.4319372437106485,
 0.4070669462197352,
 0.39566230010765957,
 0.3734936609760062,
 0.37202704428288474,
 0.3631523633480129,
 0.35021038504625274,
 0.3441531945729658,
 0.32873731835907477,
 0.263011829994628,
 0.23660076314490022,
 0.23380796795142594,
 0.22541746041428692,
 0.21891776536391688,
 0.21498910603369503,
 0.18112969573700177,
 0.15877073298005656,
 0.1282208872250961,
 0.11792300005290815,
 0.11388726220133871,
 0.1129685342219722,
 0.10976071850575135,
 0.10828619665865277,
 0.10491569588074456,
 0.09684475840976357,
 0.

In [32]:
# 选择特征IV值较小的值删除
threshold = 0.02
threshold2 = 1
data_index=[]
for i in range(len(ivl)):
    if (im_iv['IV'][i]< threshold)|(im_iv['IV'][i] > threshold2):
        data_index.append(im_iv['Var'][i])

In [34]:
# 确定需要删除的列数
len(data_index)

14

In [35]:
features_dropIV=features.drop(data_index,axis=1)
features_dropIV.head()

,low_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,repayment_capability,number_of_trans_from_2011,first_transaction_time,...,loans_product_count,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,historical_suc_fail_fee,historical_fail_rate
0,-0.300790,2.252947,-0.014732,-1.306635,-1.038731,0.095038,0.336382,-0.078710,0.590317,-1.371082,...,-0.647793,-1.157362,-1.333901,-0.848779,-0.926247,-0.886276,-0.557618,1.709946,-1.059771,-0.496026
1,-0.300790,-0.018757,-0.020529,-1.324287,-2.388106,2.791271,0.336382,-0.293537,-0.507153,-0.022965,...,0.569754,-0.067457,0.361733,-0.999475,-0.884130,-1.009926,-0.077810,-0.503430,-0.509042,-0.892044
2,0.222766,3.001860,-0.018972,-0.765300,-0.139149,-0.567970,2.183803,-0.125715,-0.606923,0.650285,...,-0.647793,-0.748647,-0.557670,-0.108999,-0.547193,0.067599,-0.424338,1.784976,-0.641217,0.509432
3,0.222766,-0.493068,-0.018068,-1.194838,-0.813836,0.006637,-0.341006,-0.200093,0.091467,0.624676,...,-0.647793,-0.748647,-0.661167,-0.492589,-0.378724,-0.664235,0.028814,-0.953609,0.107774,0.695197
4,-0.300790,-0.368250,-0.002904,-0.259269,0.085747,-0.479569,-0.341006,-0.160827,-0.606923,-0.002073,...,-0.473858,-1.157362,-0.695666,-0.684384,-0.757779,-0.714048,0.055470,-1.009881,-0.729334,-0.931876


### xgboost特征排序
利用model.get_fscore()来获得特征得分，然后排序

In [36]:
import xgboost as xgb

In [38]:
y = train_y
X = train_x
dtrain = xgb.DMatrix(X[numerics], label=y.astype('int'))

params={
        'booster':'gbtree',
        'objective': 'rank:pairwise',
        'scale_pos_weight': float(len(y)-sum(y))/float(sum(y)),
        'eval_metric': 'auc',
        'gamma':0,
        'max_depth':3,
        'subsample':0.7,
        'colsample_bytree':0.7,
        'min_child_weight':0, 
        'eta': 0.04,
        'seed':2018
        }
    
watchlist  = [(dtrain,'train')]
model = xgb.train(params,dtrain,num_boost_round=100)
      
#get feature score
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}".format(key,value))
fs[:5]

['historical_suc_rate,83',
 'trans_fail_top_count_enum_last_1_month,78',
 'historical_fail_rate,40',
 'trans_day_last_12_month,36',
 'latest_one_month_suc,34']

In [39]:
# 将进行特征选择之后的文件保存下来
features_dropIV.to_csv('../data/dropIV.csv')